*Sign Language - Machine Learning*

---

Seminarski rad u sklopu kursa Mašinsko učenje - Matematički fakultet, Univerzitet u Beogradu

**Autori**  
Nikola Dimić i Zorana Gajić

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!kaggle datasets download -d paultimothymooney/interpret-sign-language-with-deep-learning

**Opis projekta**  
 Cilj ovog projekta jeste da se konstruiše model za klasifikaciju slika znakovnog jezika kako bismo olakšali njegovo učenje i razumevanje.

**Baza podataka**  
Baza podataka sadrži 3000 slika za svako slovo. Znakovi su slikani iz različitih uglova i pod različitim osvetljenjem, kako bi se dobio što precizniji model. Link do baze je [ovde](https://www.kaggle.com/paultimothymooney/interpret-sign-language-with-deep-learning/data).



**Neophodne biblioteke**

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers
from keras import utils

from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix

**Pretprocesiranje slika**

Slike znakovnog jezika nalaze se u datotekama razvrstane po slovima. Prilikom učitavanja svake od datoteka, slika je obrađena u skladu sa potrebama mreže koja se trenira. Tako je svaka slika smanjena ili povećana u skladu sa odgovarajućim dimenzijama. Svakoj slici dodeljena je klasa u skladu sa datotekom u kojoj se nalaze, s tim što su klase konvertovane iz slovnog u brojevni zapis pa tako su slova *A,B...Z* redom označena rednim brojevima slova u abecedi (*0,1..25*), dok su specijalni karakteri *del*, *nothing* i *space* označeni brojevima *26,27,28* respektivno. Nakon obrade slika i pridruživanja klasa, ispisane su specifikacije skupa podataka, kao i dimenzije matrica kojima su opisane slike. 

In [ ]:
def letterToNumber(letter):
    if(len(letter) == 1):
        letterNum = ord(letter.lower()) - 97
    elif(letter == 'del'):
        letterNum = 26
    elif(letter == 'nothing'):
        letterNum = 27
    elif(letter == 'space'):
        letterNum = 28
    else:
        letterNum = 29
    return letterNum

def proccessImage(imageFile, size=(200,200), color = True):
    color = 3 if color else 1
    resizedImage = resize(imageFile, (size[0], size[1], color), anti_aliasing=True)
    return np.asarray(resizedImage)

def resizeData(X, size=(200,200), color = True):
    Xnew = map(lambda img: proccessImage(img,size,color), X)
    return list(Xnew)

def logData(imageColor, imageSize, imagesPerLetter, item):
    print("Extracted data with the following specifications:")
    print("--------------------------------------------------")
    print("color: " + str(imageColor))
    print("size: " + str(imageSize))
    print("images per letter: " + str(imagesPerLetter))
    print("output shape: " + str(item.shape))

def exctractDataFromFiles(file, imageColor = True, imageSize = (200,200), imagesPerLetter='all'):
    color = 1 if imageColor else 0
    X = []
    y = []
    for folderName in os.listdir(file):
        if not folderName.startswith('.'):
            classLetter = folderName
            classNum = letterToNumber(classLetter)
            fullFolderName = file+folderName
            listOfImages = os.listdir(fullFolderName)
            numOfImages = len(listOfImages) if imagesPerLetter == 'all' else imagesPerLetter
            
            for imageName in tqdm(listOfImages[0:numOfImages]):
                fullImageName = fullFolderName + "/" + imageName
                imageFile = cv2.imread(fullImageName, color)
                resizedImage = proccessImage(imageFile, imageSize, color)   
                X.append(resizedImage)
                y.append(classNum)
    X = np.asarray(X)
    y = np.asarray(y)
    logData(imageColor, imageSize, imagesPerLetter, X[0])
    return X,y


In [ ]:
trainFolder = "../input/asl-alphabet/asl_alphabet_train/"
#testFolder =  "../input/asl-alphabet/asl_alphabet_test/"

X, y = exctractDataFromFiles(trainFolder, imageColor = False, imageSize = (50,50), imagesPerLetter = 1000)

**Prikaz slike**


In [ ]:
testImage = cv2.imread("../input/asl-alphabet/asl_alphabet_train/A/A1.jpg")
resizedTestImage = proccessImage(testImage, size=(50,50), color=True)
print(resizedTestImage.shape)
plt.imshow(resizedTestImage, cmap='gray')
plt.show()

**Pretprocesiranje ciljne promenljive**  
U sklopu transformacije ciljne promenljive je korišćeno `One-Hot encoding`, odnosno kodiranje koje kategoričkom atributu (ciljnoj promenljivoj) sa $k$ mogućih različitih vrednosti (gde je $k$ u našem slučaju vrednost 30) pridružuje $k$ novih promenljivih, a vrednostima kategoričke promenljive vrednosti indikatora.

0 &#8594; 1 0 0 0 ... 0  
1 &#8594; 0 1 0 0 ... 0  
2 &#8594; 0 0 1 0 ... 0  
...  
29 &#8594; 0 0 0 0 ... 0  

In [ ]:
# One hot encoding of target variable
numOfClasses = 30
y = to_categorical(y, numOfClasses)

**Podela podataka na trening i test skup**  
Učitani podaci se dele na trening i test skup u odnosu *3:1*, s tim što se uz pomoć naglašenog parametra *stratify* čuva udeo ciljne promenljive iz originalnog skupa podataka.

In [ ]:
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

**Početni model**  
Prvi pokušaj rešavanja problema je bila konvolutivna neuronska mreža sa arhitekturom sličnoj onoj prikazanoj na vežbama. Naime, korišćena je neuronska mreža sa propagacijom unapred, gde se dodaju sledeći slojevi:  
**1 sloj:** Konvolucioni sloj sa *16* filtera, kernelom veličine *3x3*, uokvirenjem tako da veličina izlazne slike može biti istih dimenzija i *ReLu* aktivacionom funkcijom.    
**2 sloj:** Agregacioni sloj koji vrši redukciju slojeva svođenjem blokova zadatih večina na njihove maksimalne vrednosti i veličinom bloka *2x2*.  
**3 sloj:** Konvolucioni sloj sa *32* filtera, kernelom veličine *3x3*, uokvirenjem tako da veličina izlazne slike može biti istih dimenzija i *ReLu* aktivacionom funkcijom.    
**4 sloj:** Agregacioni sloj koji vrši redukciju slojeva svođenjem blokova zadatih večina na njihove maksimalne vrednosti i veličinom bloka *2x2*.  
**5 sloj:** Konvolucioni sloj sa *64* filtera, kernelom veličine *3x3*, uokvirenjem tako da veličina izlazne slike može biti istih dimenzija i *ReLu* aktivacionom funkcijom.  
Zatim funkcija koja se koristi za transformisanje matrica u vektore *Flatten*.    
**6 sloj:** Potpuno povezan sloj sa brojem izlaznih neurona *1024* i ReLu aktivacionom funkcijom.  
Zatim tehnika regularizacije kojom isključujemo nasumično odabrane neurone i omogućavamo drugačiji protok podataka kroz mrežu *Dropout*.
**7 sloj:** Potpuno povezan sloj sa brojem izlaznih neurona *30* i *softmax* aktivacionom funkcijom. 

In [ ]:
# DOBAR MODEL ZA COLOR 
# Training model
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(numOfClasses, activation='softmax'))

In [ ]:
# Model summary per layers
model.summary()

**Mrežu ćemo trenirati prema sledećim smernicama:**  
*Funkcija greške*: Kategorička unakrsna entropija  
*Optimizator*: Adam sa korakom učenja **0.001**   
*Broj epoha*: **12**  
*Veličina paketića za treniranje* (engl. batch size): **128**  

Za evaluaciju mreže koristimo **preciznost**.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
# Train model
history = model.fit(X_train, y_train,
                    batch_size=128,
                    epochs=12,
                    verbose=1,
                    validation_split=0.2, 
                    shuffle=True)

In [ ]:
# Plotting loss/accuracy values of train and validation data during training model
def plot_loss_accuracy(epochs, history):
  plt.figure(figsize=(10, 4))

  plt.subplot(1, 2, 1)
  plt.title('Loss')
  plt.plot(np.arange(0, epochs), history.history['loss'], label='train')
  plt.plot(np.arange(0, epochs), history.history['val_loss'], label='val')
  plt.legend(loc='best')

  plt.subplot(1, 2, 2)
  plt.title('Accuracy')
  plt.plot(np.arange(0, epochs), history.history['accuracy'], label='train')
  plt.plot(np.arange(0, epochs), history.history['val_accuracy'], label='val')
  plt.legend(loc='best')

Na sledećem zajedničkom grafiku je prikazana funkcija gubitka na skupu za treniranje i validaciju, a potom i funkcija preciznosti.

In [ ]:
plot_loss_accuracy(12, history)

In [ ]:
def show_classification_report(model, X_test, y_test):
  print(classification_report(np.argmax(y_test, axis=1), model.predict_classes(X_test)))

TODO

In [ ]:
show_classification_report(model, X_test, y_test):

In [ ]:
# Evaluate model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

**Model iz naučnog rada**  
Sledeći pokušaj rešavanja datog problema je bio uz pomoć pronađenog naučnog rada koji možete pogledati [ovde](https://) .  
Iako se skupovi podataka razlikuju, pokušali smo sa predloženim modelom: korišćena je neuronska mreža sa propagacijom unapred, gde se dodaju sledeći slojevi:  
**1 sloj:** Konvolucioni sloj sa *16* filtera, kernelom veličine *2x2* i *ReLu* aktivacionom funkcijom.    
**2 sloj:** Agregacioni sloj koji vrši redukciju slojeva svođenjem blokova zadatih večina na njihove maksimalne vrednosti, veličinom bloka *2x2*, veličinom pomeraja *2x2* i uokvirenjem tako da veličina izlazne slike može biti istih dimenzija.  
**3 sloj:** Konvolucioni sloj sa *32* filtera, kernelom veličine *3x3* i *ReLu* aktivacionom funkcijom.    
**4 sloj:** Agregacioni sloj koji vrši redukciju slojeva svođenjem blokova zadatih večina na njihove maksimalne vrednosti, veličinom bloka *3x3*, veličinom pomeraja *3x3* i uokvirenjem tako da veličina izlazne slike može biti istih dimenzija.   
**5 sloj:** Konvolucioni sloj sa *64* filtera, kernelom veličine *5x5* i *ReLu* aktivacionom funkcijom.  
Zatim funkcija koja se koristi za transformisanje matrica u vektore *Flatten*.    
**6 sloj:** Potpuno povezan sloj sa brojem izlaznih neurona *128* i ReLu aktivacionom funkcijom.  
Zatim tehnika regularizacije kojom isključujemo nasumično odabrane neurone i omogućavamo drugačiji protok podataka kroz mrežu *Dropout*.
**7 sloj:** Potpuno povezan sloj sa brojem izlaznih neurona *30* i *softmax* aktivacionom funkcijom. 

In [ ]:
# DOBAR MODEL ZA GRAYSCALE
# Training model
paper_model = Sequential()
paper_model.add(Conv2D(16, (2,2), input_shape=(50,50,1), activation='relu'))
paper_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
paper_model.add(Conv2D(32, (3,3), activation='relu'))
paper_model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3), padding='same'))
paper_model.add(Conv2D(64, (5,5), activation='relu'))
paper_model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
paper_model.add(Flatten())
paper_model.add(Dense(128, activation='relu'))
paper_model.add(Dropout(0.2))
paper_model.add(Dense(numOfClasses, activation='softmax'))

In [ ]:
# Model summary per layers
paper_model.summary()

**Mrežu ćemo trenirati prema sledećim smernicama:**  
*Funkcija greške*: Kategorička unakrsna entropija  
*Optimizator*: Adam sa korakom učenja **0.001**   
*Broj epoha*: **50**  
*Veličina paketića za treniranje* (engl. batch size): **500**  

Za evaluaciju mreže koristimo **preciznost**.


In [ ]:
paper_model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
# Train model
paper_history = paper_model.fit(X_train, y_train,
                    batch_size=500,
                    epochs=50,
                    verbose=1,
                    validation_split=0.2, 
                    shuffle=True)

*Na sledećem zajedničkom grafiku je prikazana funkcija gubitka na skupu za treniranje i validaciju, a potom i funkcija preciznosti.*

In [ ]:
plot_loss_accuracy(50, paper_history)

**TODO**

In [ ]:
show_classification_report(paper_model, X_test, y_test)

In [ ]:
# Evaluate model
score = paper_model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

**VGG16 model**  
Sledeći pokušaj rešavanja datog problema je bio uz pomoć arhitekture VGG16 mreže. Korišćena je neuronska mreža sa propagacijom unapred, gde se dodaju sledeći slojevi:  
**1 sloj:** Konvolucioni sloj sa *64* filtera, kernelom veličine *3x3*, okvirenjem tako da veličina izlazne slike može biti istih dimenzija i *ReLu* aktivacionom funkcijom.  
**2 sloj:** Kao i prvi sloj.  
**3 sloj:** Agregacioni sloj koji vrši redukciju slojeva svođenjem blokova zadatih večina na njihove maksimalne vrednosti, veličinom bloka *2x2*, veličinom pomeraja *2x2*.  
**4 sloj:** Konvolucioni sloj sa *128* filtera, kernelom veličine *3x3*, okvirenjem tako da veličina izlazne slike može biti istih dimenzija i *ReLu* aktivacionom funkcijom.  
**5 sloj:** Kao i četvrti sloj.   
**6 sloj:** Kao i treći sloj.     
**7 sloj:** Konvolucioni sloj sa *256* filtera, kernelom veličine *3x3*, okvirenjem tako da veličina izlazne slike može biti istih dimenzija i *ReLu* aktivacionom funkcijom.  
**8 sloj:** Kao i sedmi sloj.  
**9 sloj:** Kao i sedmi sloj.  
**10 sloj:** Konvolucioni sloj sa *512* filtera, kernelom veličine *3x3*, okvirenjem tako da veličina izlazne slike može biti istih dimenzija i *ReLu* aktivacionom funkcijom.  
**11 sloj:** Kao i deseti sloj.  
**12 sloj:** Kao i deseti sloj.
**13 sloj:** Kao i treći sloj.  
**14 sloj:** Kao i deseti sloj.  
**15 sloj:** Kao i deseti sloj.  
**16 sloj:** Kao i deseti sloj.  
**17 sloj:** Kao i treći sloj.  
Zatim funkcija koja se koristi za transformisanje matrica u vektore *Flatten*.  
**18 sloj:** Potpuno povezan sloj sa brojem izlaznih neurona *4096* i *ReLu* aktivacionom funkcijom. 
**19 sloj:** Kao i 18. sloj.  
**20 sloj:** Potpuno povezan sloj sa brojem izlaznih neurona *30* i *softmax* aktivacionom funkcijom. 


In [ ]:
# VGG16 model
vgg16_model = Sequential()

vgg16_model.add(Conv2D(input_shape=(224,224,3), filters=64, kernel_size=(3,3), padding="same", activation="relu"))
vgg16_model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
vgg16_model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
vgg16_model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
vgg16_model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
vgg16_model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
vgg16_model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
vgg16_model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
vgg16_model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
vgg16_model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
vgg16_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vgg16_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vgg16_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vgg16_model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
vgg16_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vgg16_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vgg16_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
vgg16_model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

vgg16_model.add(Flatten())
vgg16_model.add(Dense(units=4096, activation="relu"))
vgg16_model.add(Dense(units=4096, activation="relu"))
vgg16_model.add(Dense(units=numOfClasses, activation="softmax"))

In [ ]:
# Model summary per layers
model.summary()

**Mrežu ćemo trenirati prema sledećim smernicama:**  
*Funkcija greške*: Kategorička unakrsna entropija  
*Optimizator*: Adam sa korakom učenja **0.001**   
*Broj epoha*: **100**  
*Veličina paketića za treniranje* (engl. batch size): **128**  

Za evaluaciju mreže koristimo **preciznost**.

In [ ]:
vgg16_model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=128,
                    epochs=100,
                    verbose=1,
                    validation_split=0.2, 
                    shuffle=True)

*Na sledećem zajedničkom grafiku je prikazana funkcija gubitka na skupu za treniranje i validaciju, a potom i funkcija preciznosti.*

In [ ]:
plot_loss_accuracy(100, history)

**TODO**

In [ ]:
show_classification_report(vgg16_model, X_test, y_test)

In [ ]:
# Evaluate model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])